<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/adv_cv/class_2/10%20-%20%20Laborat%C3%B3rio/C3_W2_Lab_1_Simple_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Visão computacional avançada com TensorFlow](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow?specialization=tensorflow-advanced-techniques) de [Laurence Moroney](https://laurencemoroney.com/) e [Andrew Ng](https://www.coursera.org/instructor/andrewng) , [DeepLearning.AI](https://www.deeplearning.ai/)

# Detecção simples de objetos no Tensorflow

Neste laboratório, você aprenderá a usar os modelos de detecção de objetos disponíveis no [Tensorflow Hub](https://www.tensorflow.org/hub).

Nas seções a seguir, você vai:
* explorar o Tensorflow Hub para modelos de detecção de objetos
* carregar os modelos em seu espaço de trabalho
* pré-processar uma imagem para inferência 
* executar a inferência nos modelos e inspecionar a saída

## Importações

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

2024-04-29 09:49:17.637757: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-29 09:49:17.668609: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 09:49:17.817483: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 09:49:17.817519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 09:49:17.847670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

### Baixar o modelo do Tensorflow Hub

O Tensorflow Hub é um repositório de modelos de aprendizado de máquina treinados que você pode reutilizar em seus próprios projetos. 
- Você pode ver os domínios cobertos [aqui](https://www.kaggle.com/models?framework=tensorFlow2&publisher=tensorflow) e suas subcategorias. 
- Para este laboratório, você deverá examinar a subcategoria[`image-object-detection`](https://www.kaggle.com/models?task=17074&framework=tensorFlow2&publisher=tensorflow). 
- Você pode selecionar um modelo para ver mais informações sobre ele e copiar o URL para fazer o download em seu espaço de trabalho. 
- Utilizaremos um [inception resnet version 2](https://www.kaggle.com/models/tensorflow/faster-rcnn-inception-resnet-v2)
- Você também pode modificar a célula a seguir para escolher o outro modelo: [ssd mobilenet version 2](https://www.kaggle.com/models/tensorflow/ssd-mobilenet-v2)

In [2]:
# você pode trocar as linhas comentadas aqui para escolher o outro modelo

# Início da versão 2 do resnet
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"


# Em vez disso, você pode escolher a versão 2 do ssd mobilenet e comparar os resultados
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

#### Carregar o modelo

Em seguida, você carregará o modelo especificado pelo `module_handle`.
- Pode levar alguns minutos para carregar o modelo.

In [3]:
model = hub.load(module_handle)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
2024-04-29 09:49:52.289082: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-29 09:49:52.291285: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


#### Escolha a assinatura padrão

Alguns modelos no hub do Tensorflow podem ser usados para diferentes tarefas.

Portanto, a documentação de cada modelo deve mostrar qual *assinatura* usar ao executar o modelo. 
- Se quiser ver se um modelo tem mais de uma assinatura, você pode fazer algo como `print(hub.load(module_handle).signatures.keys())`.

Aqui os modelos que você usará têm apenas a assinatura `default`, portanto, você não precisa se preocupar com outros tipos.

In [4]:
# Dê uma olhada nas assinaturas disponíveis para esse modelo específico
model.signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction () -> Dict[['detection_class_labels', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None)], ['detection_scores', TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)], ['detection_boxes', TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)], ['detection_class_entities', TensorSpec(shape=(None, 1), dtype=tf.string, name=None)], ['detection_class_names', TensorSpec(shape=(None, 1), dtype=tf.string, name=None)]] at 0x73EF1A79A450>}))

Escolha a assinatura "padrão" para seu detector de objetos.

- Para modelos de detecção de objetos, sua assinatura "padrão" aceitará um lote de tensores de imagem e produzirá um dicionário que descreve os objetos detectados, que é o que você deseja aqui.

In [5]:
detector = model.signatures['default']

### download_and_resize_image

Essa função faz o download de uma imagem especificada por uma determinada "url", pré-processa-a e, em seguida, salva-a no disco.

In [6]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Obtém uma imagem on-line, redimensiona-a e salva-a localmente.
    
    Args:
        url (string) -- link para a imagem
        new_width (int) -- tamanho em pixels usado para redimensionar a largura da imagem
        new_height (int) -- tamanho em pixels usado para redimensionar o comprimento da imagem
        
    Retorna:
        (string) -- caminho para a imagem salva
    '''
    
    
    # cria um arquivo temporário que termina com ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # abre o URL fornecido
    response = urlopen(url)
    
    # lê a imagem obtida do URL
    image_data = response.read()
    
    # Coloca os dados da imagem no buffer de memória
    image_data = BytesIO(image_data)
    
    # abre a imagem
    pil_image = Image.open(image_data)

    # Redimensiona a imagem. Será cortada se a proporção for diferente.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.Resampling.LANCZOS)
    
    # converte para o espaço de cores RGB
    pil_image_rgb = pil_image.convert("RGB")
    
   # salva a imagem no arquivo temporário criado anteriormente
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Imagem baixada para %s." % filename)
    
    return filename

### Baixar e pré-processar uma imagem

Agora, usando `download_and_resize_image`, você pode obter uma imagem de amostra on-line e salvá-la localmente. 
- Forneci uma URL para você, mas fique à vontade para escolher outra imagem para passar pelo detector de objetos.
- Você pode usar a largura e a altura originais da imagem, mas fique à vontade para modificá-la e ver os resultados obtidos.

In [7]:
# Você pode escolher um URL diferente que aponte para uma imagem de sua escolha
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# Faça o download da imagem e use a altura e a largura originais
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Imagem baixada para /tmp/tmptwviu7k4.jpg.


### run_detector

Essa função receberá o modelo de detecção de objetos `detector` e o caminho para uma imagem de amostra e, em seguida, usará esse modelo para detectar objetos e exibir suas categorias de classe previstas e caixas de detecção.
- run_detector usa `load_image` para converter a imagem em um tensor.

In [8]:
def load_img(path):

    img = tf.io.read_file(path)
    
    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)
    
    return img


def run_detector(detector, path):
    '''
    Carrega uma imagem JPEG e a converte em um tensor.
    
    Args:
        path (string) -- caminho para uma imagem JPEG salva localmente
    
    Retorna:
        (tensor) -- um tensor de imagem
    '''
    
    # lê o arquivo
    img = load_img(path)

    # Adicionar uma dimensão de lote na frente do tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    
    # Executar a inferência usando o modelo
    result = detector(converted_img)

   # Salvar os resultados em um dicionário
    result = {key:value.numpy() for key,value in result.items()}

    # imprimir resultados
    print("Encontrados %d objetos." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])


### Executar inferência na imagem

Você pode executar seu detector chamando a função `run_detector`. Isso imprimirá o número de objetos encontrados seguido de três listas: 

* As pontuações de detecção de cada objeto encontrado (ou seja, o grau de confiança do modelo), 
* As classes de cada objeto encontrado, 
* As caixas delimitadoras de cada objeto

Você verá como sobrepor essas informações à imagem original nas próximas seções e na atividade avaliativa!

In [9]:
# Executa o modelo de detecção de objetos e imprime informações sobre os objetos encontrados
run_detector(detector, downloaded_image_path)

W0000 00:00:1714384263.948904     751 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "CropAndResize" attr { key: "T" value { type: DT_FLOAT } } attr { key: "extrapolation_value" value { f: 0 } } attr { key: "method" value { s: "bilinear" } } inputs { dtype: DT_FLOAT shape { dim { size: -2484 } dim { size: -2485 } dim { size: -2486 } dim { size: 1088 } } } inputs { dtype: DT_FLOAT shape { dim { size: -105 } dim { size: 4 } } } inputs { dtype: DT_INT32 shape { dim { size: -105 } } } inputs { dtype: DT_INT32 shape { dim { size: 2 } } value { dtype: DT_INT32 tensor_shape { dim { size: 2 } } int_val: 17 } } device { type: "CPU" vendor: "GenuineIntel" model: "111" frequency: 2496 num_cores: 16 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 49152 l2_cache_size: 1310720 l3_cache_size: 20971520 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { dim { size: 

Encontrados 100 objetos.
[0.6544857  0.61145216 0.6042279  0.59263206 0.5921869  0.5804905
 0.5514051  0.4946694  0.47515717 0.47342193 0.4399597  0.4148504
 0.4062956  0.3982892  0.3976523  0.37621033 0.3727929  0.36574817
 0.35260627 0.33274764 0.30428666 0.2727669  0.26864883 0.25777176
 0.25290748 0.24612032 0.23403813 0.20342915 0.18229356 0.18045719
 0.17571236 0.16435125 0.15849988 0.15666044 0.15470858 0.15452752
 0.1492493  0.13340679 0.12948358 0.12649727 0.12044238 0.11767314
 0.1135606  0.11114759 0.11100309 0.10914911 0.10604025 0.08940552
 0.08598208 0.08280208 0.08104547 0.07806066 0.07760316 0.07628672
 0.07546848 0.07444185 0.0742717  0.07204849 0.07177564 0.07102229
 0.07032698 0.06809634 0.06304505 0.06285914 0.06270956 0.06223947
 0.05882098 0.05814969 0.05795737 0.05787598 0.05462348 0.05274333
 0.05133707 0.04826561 0.04708421 0.04682778 0.04495209 0.04405119
 0.04360714 0.04113463 0.04109957 0.03968599 0.03934993 0.03912783
 0.03879511 0.03878588 0.03739639 0.036